# Requirements

## Python

In [1]:
import sys


python_version = (
    f"{sys.version_info.major}."
    f"{sys.version_info.minor}."
    f"{sys.version_info.micro}"
)
print(f"Python version {python_version}")

Python version 3.8.12


Choose any following methods to install dependcies
* poetry (recommend) <br/><br/>
 `poetry install` <br/><br/>
* pip <br/><br/>
`pip install -r requirements.txt`

## IPFS CLI

Follow this URL https://docs.ipfs.io/install/command-line/#official-distributions

# Import modules

In [2]:
import brownie
from brownie import accounts, project, config, network

# Compile smart contracts
p = project.load('.', name="pynft")
p.load_config()

# Connect to chain

In [3]:
import os

print("Available networks")
print(os.popen("brownie networks list").read())

Available networks
Brownie v1.17.0 - Python development framework for Ethereum

The following networks are declared:

Ethereum
  ├─Mainnet (Infura): mainnet
  ├─Ropsten (Infura): ropsten
  ├─Rinkeby (Infura): rinkeby
  ├─Goerli (Infura): goerli
  ├─Kovan (Infura): kovan
  ├─mainnet-fork-2: mainnet-fork-2
  ├─mainnet-fork-3: mainnet-fork-3
  ├─mainnet-fork-4: mainnet-fork-4
  └─mainnet-fork-5: mainnet-fork-5

Ethereum Classic
  ├─Mainnet: etc
  └─Kotti: kotti

Arbitrum
  └─Mainnet: arbitrum-main

Binance Smart Chain
  ├─Testnet: bsc-test
  └─Mainnet: bsc-main

Fantom Opera
  ├─Testnet: ftm-test
  └─Mainnet: ftm-main

Harmony
  └─Mainnet (Shard 0): harmony-main

Polygon
  ├─Mainnet (Infura): polygon-main
  └─Mumbai Testnet (Infura): polygon-test

XDai
  ├─Mainnet: xdai-main
  └─Testnet: xdai-test

Eherium
  └─ganache-local: ganache-local

Etherium
  └─ganache-local-2: ganache-local-2

cronos
  └─cronos-testnet: cronos-testnet

Development
  ├─Ganache-CLI: development
  ├─Geth Dev: geth-d

## Connect

In [4]:
NETWORK = "rinkeby"
# network.connect(NETWORK) # Main net fork
# network.connect(NETWORK) # Main net
network.connect(NETWORK) # Test net rinkeby

# Account

In [5]:
!brownie accounts list

Brownie v1.17.0 - Python development framework for Ethereum

Found 4 accounts:
 ├─eth-main: 0xf6EfbD8142A18E741360b41301eDFdbD2719D03C
 ├─prem: 0xedEa45C169753E1af9bf1cb1c374Ca83EBd0d6BE
 ├─test: 0xA3c11aDE2244474EB2F8167921216AB6541F9bA8
 └─test_2: 0xA3c11aDE2244474EB2F8167921216AB6541F9bA8


In [6]:
if NETWORK == "mainnet" and False:
    from scripts.helpful_script import get_account

    account = get_account()

else:
    from brownie import accounts

    accounts.load("eth-main") # Main net
    account = accounts[0]


Enter password for "eth-main": ········


In [7]:
print(account)

0xf6EfbD8142A18E741360b41301eDFdbD2719D03C


# Dump variables

In [8]:
for key, value in p.__dict__.items():
    print(key)
    brownie.__dict__[key] = p.__dict__[key]

_path
_envvars
_structure
_build_path
_name
_active
_sources
_build
_compiler_config
interface
_containers
PYNFT
ERC721
Strings
__all__
_namespaces


# Deploy contracts

In [10]:
from brownie import PYNFT


pynft = PYNFT.deploy({"from": account})

Transaction sent: 0xa2a16103b7ce13a0015421ea2cfdbb3319f6617d16e65c6175a50ea57a09bfab
  Gas price: 1.705944452 gwei   Gas limit: 1419476   Nonce: 29
  PYNFT.constructor confirmed   Block: 10057857   Gas used: 1290433 (90.91%)
  PYNFT deployed at: 0xeee906fAE06C0456b57A11cBd5Cec4628fBDAAf2



# Upload file to IPFS

## Assets

In [11]:
prefix = "assets/"


def upload_to_ipfs(filepath: str) -> str:
    return os.popen(f"ipfs add -q {filepath}").read().strip()

def is_an_image(filename: str):
    """
    Description:
        Check if a file is an image
    paramgs:
        filename:
            (str) filename, should be with its extension
    returns:
        (str) qid of ipfs
    """
    for extention in [".png", "jpg"]:
        if filename.endswith(extention):
            return True
    return False

def set_ipfs_share_link(prefix: str, asset: str) -> str:
    """
    description:
        Run IPFS upload process and return share link URL
    params:
        prefix:
            (str) Local prefix file
        asset:
            (str) Filename
    returns:
        (str) share link url
    """
    IPFS_PREFIX = "https://ipfs.io/ipfs"
    qid = upload_to_ipfs(f"{prefix}{asset}".replace(" ", "\ "))
    asset = asset.replace(" ", "-")
    return f"{IPFS_PREFIX}/{qid}?filename={asset}"



class Attribute:
    pass


attribute = Attribute()
for asset in os.listdir(prefix):
    if asset.startswith(".") or asset == "metadata.json":
        # Skip
        continue
    ipfs_share_link = set_ipfs_share_link(prefix, asset)
    if is_an_image(asset):
        setattr(attribute, "image_url", ipfs_share_link)
    else:
        setattr(attribute, "external_url", ipfs_share_link)
    print(ipfs_share_link)

https://ipfs.io/ipfs/Qmda8x41dArU36LKveHzSN3NPJjWxFgyeNNLM8uUMqtMNr?filename=my-first-python-code.png
https://ipfs.io/ipfs/QmUoGY6GbvzSQi2gP7YHtVMRocNW1Xs16ZUHxWcTgWHwJb?filename=fruit-list.py


# Set up metadata

## Render metadata from template

In [12]:
metadata = {
    "description": """It was in 2015, I was an engineer intern.
    
I was assigned to code in Python for satellite signal processing.

At that time, I had no experience with it, but I had completed C#, C and Matlab courses.

It inspired me to transfer from electrical to software engineer career path.

Now, I partially succeed in software engineering, machine learning and A.I implementation.

This NFT determines where I started.

Moreover, I minting this NFT with my Solidity code I created by myself.

No, it's not my first Solidity project :D

But it's my first Solidity project deployed in `production`.

Which means this is the starting point for my smart contract development.

Smart contract repository is located in https://github.com/batprem/pynft
""", 
    "external_url": attribute.external_url, 
    "image": attribute.image_url, 
    "name": "My first Python code",
    "attributes": [], 
}

## Save metadata into JSON

In [13]:
import json


metadata_filename = "metadata.json"

with open(f"assets/{metadata_filename}", "w") as json_file:
    json.dump(metadata, json_file)
    
metadata_share_link = set_ipfs_share_link(prefix, metadata_filename)

# Assign to smart contract

## Mint

In [14]:
PYNFT[-1].createCollectable({"from": account}).wait(1)

Transaction sent: 0xce49cf20c4a9a7b70408ef8ccd9449ef6001c7a300f679af65d29cee4fe851f2
  Gas price: 1.721482323 gwei   Gas limit: 102072   Nonce: 30
  PYNFT.createCollectable confirmed   Block: 10057863   Gas used: 92793 (90.91%)

  PYNFT.createCollectable confirmed   Block: 10057863   Gas used: 92793 (90.91%)



## Set URI

In [16]:
PYNFT[-1].setBaseURI(metadata_share_link, {"from": account}).wait(1)

Transaction sent: 0x516f50b0b592ffbfe76dbc4a91595b16bc96c40be05491444d2eeb0c8a954321
  Gas price: 1.653401301 gwei   Gas limit: 38577   Nonce: 32
  PYNFT.setBaseURI confirmed   Block: 10057867   Gas used: 35070 (90.91%)

  PYNFT.setBaseURI confirmed   Block: 10057867   Gas used: 35070 (90.91%)



In [17]:
PYNFT[-1].tokenURI(0)

'https://ipfs.io/ipfs/QmQBZ6VjMeDphzuTGQxxSst7565yjgwvvs8GHtwbayw6hU?filename=metadata.json'

In [18]:
PYNFT[-1].tokenCounter()

1

## Reference

* https://www.youtube.com/watch?v=M576WGiDBdQ&t=8373s